# Marathons From CSV:

## This is going to be EDA for the marathon data from the CSV files. I'm doing this because we had issues webscraping. Not sure if this will be my primary notebook

In [1]:
#Importing Packages

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
full_2016_df = pd.read_csv('data/full_2016.csv')

In [3]:
full_2017_df = pd.read_csv('data/full_2017.csv')

In [4]:
full_2018_df = pd.read_csv('data/full_2018.csv')

In [5]:
full_2019_df = pd.read_csv('data/full_2019.csv')

In [6]:
half_2016_df = pd.read_csv('data/half_2016.csv')

In [7]:
half_2017_df = pd.read_csv('data/half_2017.csv')

In [8]:
half_2018_df = pd.read_csv('data/half_2018.csv')

In [9]:
half_2019_df = pd.read_csv('data/half_2019.csv')

# Next Step: getting year column into DFs

In [10]:
#doing it for all other DFs
full_2016_df['Year'] = 2016
full_2017_df['Year'] = 2017
full_2018_df['Year'] = 2018
full_2019_df['Year'] = 2019
half_2016_df['Year'] = 2016
half_2017_df['Year'] = 2017
half_2018_df['Year'] = 2018
half_2019_df['Year'] = 2019

# Creating concatenated dataframes for All Fulls and All Halfs

In [11]:
all_full_df = pd.concat([full_2016_df, full_2017_df, full_2018_df, full_2019_df], axis=0)

In [12]:
all_half_df = pd.concat([half_2016_df, half_2017_df, half_2018_df, half_2019_df], axis=0)

# I'm gonna drop all of the finish times of zero, since I don't think I'm interested in looking at non-finishers. I'll name the DFs something else in case I change my mind. 

In [13]:
all_full_finishers_df = all_full_df[all_full_df.Time != '00:00:00' ]

In [14]:
all_half_finishers_df = all_half_df[all_half_df.Time != '00:00:00' ]

# Converting 'Time' to a timedelta variable. Because I'm playing with the regression model in the 2018 dataframe, I'm going to change the name to "timedelta" in that dataframe. I won't bother with renaming the other DFs for now, since I'm just playing with the 2018 full marathon.

In [15]:
all_full_finishers_df['Time'] = pd.to_timedelta(all_full_finishers_df['Time'])

C:\Users\dingw\Anaconda3_new\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
full_2018_df['TimeDelta'] = pd.to_timedelta(full_2018_df['Time'])

In [17]:
all_half_finishers_df['Time'] = pd.to_timedelta(all_half_finishers_df['Time'])

C:\Users\dingw\Anaconda3_new\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# # Markdown: I don't know what's going on with this '99999' code- we don't really have a great way of validating the other observations. Obviously the really fast times (faster than the winners) are misplaced - but we also have some slow times coded 99999. 

# We also have some half-marathon times coded 99999. I'm gonna wait for some gossip on this one, and proceed with my coding efforts as if everything was clean. 

# There's some misplaced data in our 'full' dataframe. We know that some of them are half marathon data - are all of them half marathon data?

In [18]:
all_full_finishers_df_subgroup = all_full_finishers_df[all_full_finishers_df.Overall == 99999 ]

In [19]:
all_half_finishers_df_subgroup = all_half_finishers_df[all_half_finishers_df.Overall == 99999 ]

In [20]:
all_full_finishers_df.Name.value_counts()

Richard Brooks         5
Denise Sauriol         4
Jonathan Lister        4
Dallas Smith           4
Angela Whitley         4
Kelsey Otten           4
Erik Stephan           4
Melissa Boothe         4
Carl Dunphy            4
Dean Bowden            4
Jane Tao               4
Scott Wietecha         4
Lauren Terrell         4
Justin Milam           4
Larry Epps             4
Larry Phillips         4
William Moore          4
Luke Tuttle            4
Tien Le                4
Lauren Johnston        4
James Smith            4
Eddie Godwin           4
Paul Bradbury          4
Frank Sames            4
Brian Wright           4
Guido Hussels          4
Amanda Staggs          4
Deanna Alexander       4
Alan Pettit            4
Jason Lee              4
                      ..
Tanya Truszkowski      1
Nicole Mancini         1
Heini Tallent          1
Takae Miller           1
Katherine Pangburn     1
Erin Welch             1
Paul Geffken           1
Candice Appleby        1
Stella Christoforou    1


In [21]:
all_full_finishers_df.Time.describe()

count                      9705
mean     0 days 04:48:27.211334
std      0 days 00:45:46.974559
min             0 days 01:26:11
25%             0 days 04:15:30
50%             0 days 04:49:04
75%             0 days 05:22:04
max             0 days 07:00:00
Name: Time, dtype: object

In [22]:
all_half_finishers_df_subgroup.head(16)

,Overall,Bib,Name,Time,Year
18947,99999,25610,Amy Saffell,02:42:13,2016
19295,99999,37147,Nancy Armstrong,02:11:31,2016
20651,99999,28465,Vickey Boston,01:41:13,2016
18300,99999,32405,Melissa Peplow,02:53:08,2017
19191,99999,33482,Mark Whitlock,03:03:11,2017
19371,99999,7456,Todd Walter,02:10:30,2017
19396,99999,35109,Sara Moreland,02:44:23,2017
20200,99999,20613,Brian Thompson,03:36:21,2017
20284,99999,26542,Amy Saffell,02:45:15,2017
20471,99999,31611,Sara Solomon,02:32:39,2017


In [23]:
all_half_finishers_df_subgroup.Time.describe()

count                        16
mean            0 days 02:26:02
std      0 days 00:35:33.481880
min             0 days 01:27:06
25%      0 days 02:03:10.750000
50%             0 days 02:37:26
75%      0 days 02:45:08.250000
max             0 days 03:36:21
Name: Time, dtype: object

In [24]:
all_full_finishers_df.sort_values(by=['Time'])

,Overall,Bib,Name,Time,Year
3275,99999,27024,James Cook,01:26:11,2017
0,1,30001,Travis Peruski,01:37:54,2019
2469,99999,1202,Andre Mcdonald,02:08:22,2017
2811,99999,15476,Ashley Grindstaff,02:19:14,2017
3362,99999,1163,Reon Holloway,02:22:41,2017
3359,99999,4109,Bruce Kesselring,02:24:09,2016
2678,99999,2104,George Spilich,02:24:21,2017
2593,99999,15071,Kenneth Moffat,02:25:00,2017
0,1,1,Scott Wietecha,02:25:42,2016
0,1,1,Scott Wietecha,02:28:16,2018


In [25]:
all_full_finishers_df_subgroup.Year.value_counts()

2017    129
2016     26
Name: Year, dtype: int64

# Current DQ issues: we have times faster than Scott in 2016, 2017 and 2019. I'll do all of my single-marathon experimentation with the 2018 marathon for now, since there are no abnormally fast finishes. 

# It also might be fine to use all of the half-marathon finishers as well, since there are no immediately obvious DQ issues. 

## Let's try doing a timediff variable for the 2018 full marathon

In [26]:
full_2018_df.dtypes

Overall                int64
Bib                    int64
Name                  object
Time                  object
Year                   int64
TimeDelta    timedelta64[ns]
dtype: object

In [27]:
time_diff = full_2018_df.TimeDelta.diff()

In [28]:
time_diff_stringB = full_2018_df.TimeDelta.diff().astype(str)

In [29]:
time_diff_stringB=pd.DataFrame(time_diff_stringB)

In [30]:
time_diff=pd.DataFrame(time_diff)

In [31]:
full_2018_df['time_diff']=time_diff

In [32]:
full_2018_df['time_diff_stringB']=time_diff_stringB

In [33]:
full_2018_df['Hours'] = full_2018_df.Time.str[0:2].astype(float)

In [34]:
full_2018_df['Minutes'] = full_2018_df.Time.str[3:5].astype(float)

In [35]:
full_2018_df['Seconds'] = full_2018_df.Time.str[6:8].astype(float)

In [36]:
full_2018_df['Number_Of_Minutes'] = ((full_2018_df.Hours * 60)) + ((full_2018_df.Minutes)) + ((full_2018_df.Seconds)/(60))

In [37]:
full_2018_df.dtypes

Overall                        int64
Bib                            int64
Name                          object
Time                          object
Year                           int64
TimeDelta            timedelta64[ns]
time_diff            timedelta64[ns]
time_diff_stringB             object
Hours                        float64
Minutes                      float64
Seconds                      float64
Number_Of_Minutes            float64
dtype: object

In [51]:
full_2018_df.tail()

,Overall,Bib,Name,Time,Year,TimeDelta,time_diff,time_diff_stringB,Hours,Minutes,Seconds,Number_Of_Minutes
2110,99999,31683,Aubrey Spaeth,00:00:00,2018,0 days,0 days 00:00:00,0 days 00:00:00.000000000,0.0,0.0,0.0,0.0
2111,99999,33654,Jeffrey Ekkart,00:00:00,2018,0 days,0 days 00:00:00,0 days 00:00:00.000000000,0.0,0.0,0.0,0.0
2112,99999,31170,Austin Perkins,00:00:00,2018,0 days,0 days 00:00:00,0 days 00:00:00.000000000,0.0,0.0,0.0,0.0
2113,99999,31515,Tony Quernemoen,00:00:00,2018,0 days,0 days 00:00:00,0 days 00:00:00.000000000,0.0,0.0,0.0,0.0
2114,99999,30482,David Ramos,00:00:00,2018,0 days,0 days 00:00:00,0 days 00:00:00.000000000,0.0,0.0,0.0,0.0


# This is where I'm trying to make the numerical time differential.

I think the issue is that we have NaT for the first place entry, so I'm going to replace it with zero. I can't drop the value, because the first entry will always have a NaT time differential.

Dammit Scott! It's not always about you.

In [45]:
full_2018_df.loc[0,'time_diff_stringB']= '0 days 00:00:00.000000000'

In [47]:
full_2018_df.loc[0,'time_diff']= '00:00:00'

In [49]:
full_2018_df.dtypes

Overall                        int64
Bib                            int64
Name                          object
Time                          object
Year                           int64
TimeDelta            timedelta64[ns]
time_diff                     object
time_diff_stringB             object
Hours                        float64
Minutes                      float64
Seconds                      float64
Number_Of_Minutes            float64
dtype: object

In [50]:
full_2018_df['time_diff_hours'] = full_2018_df.time_diff_stringB.str[7:9].astype(float)

ValueError: could not convert string to float: ' +'

# Markdown: I'm shutting down for the day, but I'll revisit here tomorrow. 

In [ ]:
full_2018_df['time_diff_minutes'] = full_2018_df.time_diff_stringB.str[10:12].astype(float)

In [ ]:
full_2018_df['time_diff_seconds'] = full_2018_df.time_diff_stringB.str[13:15].astype(float)

In [ ]:
full_2018_df['time_diff_thingy'] = ((full_2018_df.time_diff_hours * 60)) + ((full_2018_df.time_diff_minutes)) + ((full_2018_df.time_diff_seconds)/(60))

In [ ]:
full_2018_df.head()

In [ ]:
full_2018_df.dtypes

In [ ]:
full_2018_df.time_diff.describe()

In [ ]:
full_2018_df.loc[full_2018_df['time_diff'].idxmin()]

In [ ]:
full_2018_df = full_2018_df[full_2018_df.Time != '00:00:00' ]

# OK let's try doing the conversion to # of seconds

In [ ]:
full_2018_df['Hours'] = full_2018_df.Time.str[0:2].astype(float)

In [ ]:
full_2018_df['Minutes'] = full_2018_df.Time.str[3:5].astype(float)

In [ ]:
full_2018_df['Seconds'] = full_2018_df.Time.str[6:8].astype(float)

In [ ]:
full_2018_df['time_diff_string'] = full_2018_df.time_diff.astype(str)

In [ ]:
full_2018_df.head()

# Now I do the same conversion for timediff

In [ ]:
full_2018_df['Diff_Hours'] = full_2018_df.time_diff_string.str[0:2].astype(float)

In [ ]:
full_2018_df['Diff_Minutes'] = full_2018_df.time_diff.str[3:5].astype(float)

In [ ]:
full_2018_df['Diff_Seconds'] = full_2018_df.time_diff.str[6:8].astype(float)

In [ ]:
full_2018_df.head()

In [ ]:
full_2018_df.dtypes

In [ ]:
#full_2018_df['Number_Minutes'] = ((full_2018_df.Hours * 60)) + (full_2018_df.Minutes * 1) + (full_2018_df.Seconds * (1/60)))

# Note: I think it's functional now: I'm gonna comment out the A B and C things and see if it just works as a formula

In [ ]:
#full_2018_df['Number_Minutes_A'] = ((full_2018_df.Hours * 60))

In [ ]:
#full_2018_df['Number_Minutes_B'] =  ((full_2018_df.Minutes))

In [ ]:
#full_2018_df['Number_Minutes_C'] = ((full_2018_df.Seconds)/(60))

In [ ]:
full_2018_df.dtypes

In [ ]:
full_2018_df['Number_Of_Minutes'] = ((full_2018_df.Hours * 60)) + ((full_2018_df.Minutes)) + ((full_2018_df.Seconds)/(60))

In [ ]:
full_2018_df['Diff_Number_Of_Minutes'] = ((full_2018_df.Diff_Hours * 60)) + ((full_2018_df.Diff_Minutes)) + ((full_2018_df.Diff_Seconds)/(60))

In [ ]:
full_2018_df.head()

In [ ]:
## Without a constant


X = full_2018_df.Overall
y = full_2018_df.Bib

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

# This was just my example regression to see if regression worked- but R-Squared is actually decent. 

In [ ]:
full_2018_df.dtypes